In [ ]:
%matplotlib inline
import os
os.environ['THEANO_FLAGS'] = 'device=cpu,compiledir=.cpu'
#os.environ['THEANO_FLAGS'] = 'device=cpu,compiledir=.cpu'

import sys
sys.path.append('..')
import matplotlib.pyplot as plt
from tools.common import disp_grid
from tools.common import preprocess_gen_data
from IPython.display import Image
from skimage.io import imsave
from tools.common import load_model
from keras.models import model_from_json
import os
import joblib
import numpy as np
from lightjob.cli import load_db
#from skimage.io import imread
from tools.common import find_generation_job, find_training_job, to_generation, compute_sample_objectness, resize_set
from IPython.display import Image, display
from lightjob.cli import load_db
from lightjob.db import SUCCESS
import pandas as pd
import h5py
import cv2

from skimage.transform import resize

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from joblib.parallel import delayed, Parallel
from sklearn.neighbors import NearestNeighbors


def disp_pred(pred, h=20, w=100, y0=25, fontsize=0.9):
    nb_classes = len(pred)
    img = np.zeros((nb_classes * h, w, 3))
    cv2.rectangle(img,(0,0),(img.shape[1],img.shape[0]),(0.3,0.3,0.3),3)
    pad = 0
    col = [[0, 0, 1]] * nb_classes
    for i in range(nb_classes):
        p = int(w * pred[i])
        img[i*(h+pad):i*(h+pad)+h, 0:p, :] = col[i]
    font = cv2.FONT_HERSHEY_SIMPLEX
    for i in range(nb_classes):
        cv2.putText(img,str(i),(10,y0+h*i), font, fontsize,(1,1,1),1,cv2.LINE_AA)
        cv2.rectangle(img,(0, i*(h+pad)),(w, i*(h+pad)+h),(0.3,0.3,0.3),2)

    return img

def disp_grid_with_pred(img, preds, h=30, w=200, y0=25, fontsize=0.9):
    img = resize_set(img, 100, 100)
    img_ = np.zeros((img.shape[0], 3, img.shape[2]*2, img.shape[3]*2))
    img_[:, :, 0:img.shape[2], 0:img.shape[3]] = img[:, :, :, :]
    for i, pr in enumerate(preds):
        text= disp_pred(pr, h=h, w=w, y0=y0, fontsize=fontsize)
        text = resize(text, (img.shape[2], img.shape[3]))
        text = text.transpose((2, 0, 1))
        img_[i, :, 0:img.shape[2], img.shape[3]:] = text
    img_ = disp_grid(img_)
    return img_

from tools.common import resize_set
import shutil
from tools.common import compute_objectness, compute_objectness_renyi, compute_sample_objectness, compute_sample_objectness_renyi
def load_h5(filename):
    f = h5py.File(filename)
    X = f['X']
    nb = f.attrs['nb']
    X = X[0:nb]
    X = np.array(X)
    X = X[np.isnan(X).sum(axis=(1,2,3))==0]
    return X

In [ ]:
model_folder = '../tools/models/external/fonts_and_digits'
arch = (open(os.path.join(model_folder, 'model.json'))).read()
#arch = arch.replace('softmax', 'linear')
model = model_from_json(arch)
model.load_weights(os.path.join(model_folder, 'model.pkl'))

## checking all generated

In [ ]:
#X = load_h5('../exported_data/datasets/jobset83.hdf5')
#X = load_h5('../exported_data/datasets/jobset76.hdf5')

In [ ]:
names = ['aa', 'gan']
load_data = [
    #lambda:load_h5('../exported_data/datasets/jobset83.hdf5')[500000:1000000],
    lambda:joblib.load('../jobs/results/517302ec5da070804fd729e9bd44fdc8/images.npz')[:,-1],
    lambda:joblib.load('/home/mcherti/dcgan/jobs/5a91ac06f1c52a41c2d43cb6423404d5/gen.npz')[0:1000]
]

In [ ]:
%%time
preds_per_dataset = {}
datasets = {}
for name, d in zip(names, load_data):
    X = d()
    datasets[name] = X
    batch_size = 2048
    preds = []
    for i in range(0, len(X), batch_size):
        x = X[i:i+batch_size]
        preds.append(model.predict(x))
    preds = np.concatenate(preds, axis=0)
    preds_per_dataset[name] = preds

In [ ]:
print(preds_per_dataset['gan'].shape)

In [ ]:
import seaborn as sns
from collections import defaultdict

classes = '0123456789abcdefghijklmnopqrstuvwxyz'

def get_df(p, name):
    df = pd.DataFrame()
    df['argmax'] = map(lambda k:classes[k], p.argmax(axis=1))
    df['argmax_count'] = [1./len(p)] * len(p)
    df['model'] = [name] * len(p)
    return df

df_aa = get_df(preds_per_dataset['aa'], 'aa')
df_gan = get_df(preds_per_dataset['gan'], 'gan')
#df = pd.concat((df_aa, df_gan))
df = df_aa
fig = plt.figure(figsize=(20, 10))
ax = sns.barplot(x="argmax", y="argmax_count", hue='model', data=df, estimator=lambda x:x.sum(), order=classes)
plt.xlabel('class')
plt.ylabel('freq')
plt.axvline(x=9.5, c='red')

In [ ]:
#n = 'abcdefghijklmnopqrstuvwxyz'.index('c')
n = 3
indices = np.arange(len(datasets['aa']))
#indices = indices[compute_objectness(preds)>0.1]
p=preds_per_dataset['aa'][:, n]
indices = indices[np.argsort(p[indices])[::-1]]
#indices = indices[p[indices]>0.5]
img = datasets['aa'][indices]
img = disp_grid(img[0:4], border=1, bordercolor=(0.3, 0, 0))
imsave('imgs/out.png', img)
Image('imgs/out.png')

In [ ]:
nb_classes = 26
nbrows = 6
nbcols = 6
border = 1
space = 10
scores = {}
SIZE = 4
s = (28+border) * SIZE + border
MODEL = 'aa'
ims = {}
letters = {}
classes = '0123456789abcdefghijklmnopqrstuvwxyz'
img_all = np.ones((nbrows * (s+space), nbcols * (s+space)  ))
for i in range(nbrows):
    for j in range(nbcols):
        indices = np.arange(len(datasets[MODEL]))
        if i*nbcols+j>=nb_classes:
            break
        pr_all = preds_per_dataset[MODEL]
        p=preds_per_dataset[MODEL][:, i*nbcols+j]        
        indices = np.arange(len(datasets[MODEL]))
        indices = indices[np.argsort(p[indices])[::-1]]
        #indices = indices[pr_all[indices][:, 10:].sum(axis=1)>0.95]
        img = datasets[MODEL][indices[0:SIZE*SIZE]]
        scores[i*nbcols+j] = p[indices].copy()
        #print(p[indices][0:16])
        #letters[classes[i*nbcols+j]] = img[np.random.randint(0, 4)]
        #letters[classes[i*nbcols+j]] = img[manual[classes[i*nbcols+j]]]
        letters[classes[i*nbcols+j]] = img[0]
        img = disp_grid(img[0:SIZE*SIZE], border=border, bordercolor=(0.3, 0, 0), shape=(SIZE,SIZE))
        img_all[i*(s+space):i*(s+space) + s, j*(s+space):j*(s+space) + s] = img[:,:,0]
#img_all = 1 - img_all
imsave('imgs/all.png', img_all)
Image('imgs/all.png')

In [ ]:

pan = [
    "Pack my box with five dozen liquor jugs".lower(),
    "Jackdaws love my big sphinx of quartz".lower(),
    "The five boxing wizards jump quickly.".lower(),
    "How vexingly quick daft zebras jump".lower(),
    "Bright vixens jump dozy fowl quack".lower(),
    "Sphinx of black quartz, judge my vow".lower(),
]
for s in pan:
    pangram = np.zeros((28 , 28 * len(s)))
    i = 0
    for l in s:
        if l not in letters:
            i += 28
            continue
        pangram[0:28, i:i+28] = letters[l]

        i += 24
    pangram = 1 - pangram
    imsave('out.png', pangram)
    display(Image('out.png'))

## checking stats of jobs

In [ ]:
db = load_db()
jobs = db.jobs_with(state=SUCCESS, where='jobset83')
S = [j['summary'] for j in jobs]
jobs_gen = to_generation(jobs)

In [ ]:
k = 'stats.out_of_the_box_classification.fonts.objectness'
jobs_gen = to_generation(jobs)

def sort_by(jobs, k):
    indices = np.arange(len(jobs_gen))
    def key(i):
        val = db.get_value(jobs_gen[i], k, if_not_found=None)
        if val is None or np.isnan(val):
            val = -np.inf
        return val
    indices = sorted(indices, key=key)
    vals = [key(ind) for ind in indices]
    return indices[::-1], vals[::-1]

indices, vals = sort_by(jobs, k)

interesting = [
    jobs_gen[ind]['summary']
    for ind in indices[0:10]
]

In [ ]:
nb = 20
k1 = 'stats.out_of_the_box_classification.fonts.objectness'
indices, vals = sort_by(jobs, k1)
s1 = [jobs[ind]['summary'] for ind in indices[0:nb]]
k2 = 'stats.out_of_the_box_classification.letterness.diversity_count_letters_99'
indices, vals = sort_by(jobs, k2)
s2 = [jobs[ind]['summary'] for ind in indices[0:nb]]
k3 = 'stats.out_of_the_box_classification.letterness.diversity_max_letters'
indices, vals = sort_by(jobs, k3)
s3 = [jobs[ind]['summary'] for ind in indices[0:nb]]

In [ ]:
print(len(set(s1).intersection(set(s2))))

In [ ]:
#m = list((set(s1).intersection(set(s2)).intersection(set(s3))  ))
m = list(set(s1).intersection(s2))
print(m)

In [ ]:
I = 8
db.get_job_by_summary(m[I])['content']['model_params'], m[I]

In [ ]:
from tools.common import find_generation_job

In [ ]:
# Intersect on s1 and s2, top 20, AA
summaries = [
    '9c5cc9c315ad13f6e1151200f885d54a',
    'a3a97a790acf41b122a94af93834a301',
    '1b5f929796b52352a009ab37f602bfbf', # used
    '0396380c8787034efd079b32a51d9dd6', #used
    '6c761ed85005ba530a636fcc2a751c1d',#used
]
# 
gen_summ = [
    find_generation_job(s)['summary']
    for s in summaries
]
for s in gen_summ:
    print(s)

In [ ]:
for s in summaries[2:]:
    print(db.get_job_by_summary(s)['content'])

In [ ]:
field = 'stats.out_of_the_box_classification.letterness.count_fonts'
indices = np.arange(len(jobs))
for i in range(len(jobs_gen)):
    j = jobs_gen[i]
    v = db.get_value(j, field, if_not_found=None)
    #if not v:print('Misssing generation jobs from training job : {}, skip it.'.format(jobs[i]['summary']))

In [ ]:
indices = np.arange(len(jobs))
objectness = map(lambda j:db.get_value(j, field, if_not_found=np.nan), jobs_gen)
objectness = np.array(objectness)
indices = filter(lambda ind:not np.isnan(objectness[ind]), indices)
indices = sorted(indices, key=lambda i:objectness[i])
indices = indices[::-1]
images = ['../exported_data/figs/generated/jobset{:05d}/{}.png'.format(int(jobs[i]['where'][6:]), jobs_gen[i]['summary']) 
          for i in indices]
for i in range(len(images)):
    #print(jobs_gen[indices[i]]['summary'])
    shutil.copy(images[i], 'export/{:05d}.png'.format(i))
    #if i<20:
    #    display(Image(images[i], width=600, height=600))

In [ ]:
## images = joblib.load('../jobs/results/a7080dba22d7e429bf70bf4c1899dc32/images.npz')
images = images[:, -1]
print(images.sum())
print(images.shape)
images = images #/ float(images.max())
print(images.min(), images.max())
print(images.sum())
pr = model.predict(images)

In [ ]:
import numpy as np
fonts = np.load('/home/mcherti/work/data/fonts/fonts.npz')
fonts_X = fonts['X']
fonts_X = fonts_X / 255.
fx = fonts_X[0:1000]
fx = resize_set(fx, 28, 28)
fx = 1 - fx

In [ ]:
pr = model.predict(fx)

In [ ]:
def entropy(x):
    return -np.dot(x, np.log(x))

In [ ]:
pred_proba_letters = pr[:, 10:]
theta = 0.9
selected_letters = (pred_proba_letters.max(axis=1) > theta)
ent = entropy(probas_from_occurences(pred_proba_letters[selected_letters].argmax(axis=1))) / np.log(26)
print(selected_letters.mean() + ent)

In [ ]:
def compute_objectness(v):
    v = np.array(v)
    marginal = v.mean(axis=0)
    score = v * np.log(v / marginal)
    score = score.sum(axis=1).mean()
    score = np.exp(score)
    score = float(score)
    return score
compute_objectness(pr)

## objectness score

In [ ]:
objectness = compute_sample_objectness_renyi(preds)

In [ ]:
ind = np.argsort(objectness)[::-1]
X_sorted = X[ind]
preds_sorted = preds[ind]
objectness_sorted = objectness[ind]

In [ ]:
sample = np.arange(len(X))
np.random.shuffle(sample)
sample = sample[0:100]
objectness_sampled = objectness[sample]
X_sampled = X[sample]
preds_sampled = preds[sample]
X_sampled = X_sampled[np.argsort(objectness_sampled)[::-1]]
preds_sampled = preds_sampled[np.argsort(objectness_sampled)[::-1]]

In [ ]:
im = disp_grid_with_pred(X_sorted[0::1000], preds_sorted[0::1000], h=200)
imsave('imgs/out.png', im)
Image('imgs/out.png', width=800, height=800)

## tsne

In [2]:
interesting= [
"014c75c6714d739aee7259a321971e9c",
"01634d66ca2f265505c8e3c19dabfc08",
"01c78a7a5974a03c7bf9386bceea0ccc",
"0589a90fb79fab1eb30a896ab7e82157",
"075bfc928d21ecc4dbf2726d08080466",
"093d320192b53610a447eb55ae42541b",
"0a0794aad864797fe90f00f02fffcacd",
"0a3b974348b7a6b4d2540f37898f52b9",
"0a4827cdf66a65e1355a28db0338016c",
"0acf9872f8a68ffd86388175e337794b",
"0ad7ef561f6f48312cd552a1a2742a02",
"0afd2e2fe618625084d7f23f331811ee",
"0b6b873ba7dffe6f9824b9ced6184266",
"1207cc0a53862d926f35e22b822555d9",
"14ad5a8bb42e245a69bdad3d0fef4c4a",
"1676800b0388dc29a77b64ef0078c916",
"1b7d9d5c502aa2101c190fb91a5bfdea",
"1bcbb8839774e00c6e84edd345432108",
"1d3f210dc793a588638f0251cb6ea0c4",
"1dc52531190cf9129676e1903d376feb",
"1f64aa37b468f3246f240b261117694a",
"2188d0535c3fb0ed9c8a82d922629d88",
"21ec0a06557ca67574322579a2edd17e",
"27c31ab04817b8aabd726e4b735ed2e8",
"2c364593260fd6afceb7f270c72b1720",
"3246e5a1dd2b4ef900ea9dbca91450a9",
"343ed87ba5a1bb6df197f485a6d4fd4c",
"36933fb2fc340a7277a45469f8891b59",
"39c48512bcfba7adbf8981f0f528aed2",
"3a1fc241e455e5c171abb0d553cfd7a6",
"3b7538f5b5561e7628da0efc1d711be6",
"3f970e54af5bfce6a26ddf76e0ff2648",
"416b0c2bc209aae34533a19dbe21e123",
"4527d2a503622dfcfadde0707c45d09d",
"45d586ec0faf4adae82f920733a18d5c",
"48fd385467bf47bc5eefa485581331fe",
"4cfa603a7ea384786196634bff9256e3",
"4f9870c8ae68854360786e743453f929",
"517302ec5da070804fd729e9bd44fdc8",
"53e03350ee1ab88baebce3666ebbb0c6",
"542cb46231f30d1e2aac5682f1e38d53",
"55e1972878256226fb2369c19967ea78",
"561ce9963bf2d6a14779f2764f59b341",
"5c640320afe2a45bca400f4ae5df2973",
"611cfe20db4f8290196166e5ad7335a6",
"690f0fc689eecd14b7198fc34b605870",
"6aaf83165a07d97f79755fe0579a27cc",
"7327ad57b2a2e1ae7193c339e4ec0de8",
"73da00fd56f4bba3a9cf4502d045ef70",
"74481f32be588d6c783b4ef6052d8bc7",
"77aaf83e42fa16c30d09f51aff159d6e",
"77f66fa1d3ac15604d78a907f0a9a20a",
"7abf2cd29ba129a5ee2bc3a7b4274411",
"7b7311a13a7d2df8b64c9d5ea5f253e1",
"7dacf0429cb8624805444b6efb413023",
"7e528009eacf89cd993333ba568087d3",
"80b0469a717600c5c4192bad51a53fc2",
"8c3d35136bce99a3b18f97c06398f24a",
"8fd2c3f14f7454ff3d41db32c6326eba",
"907e85a66293d7c3675211f5606f09c4",
"91d32b40bff2db047c5dc159efcfc039",
"9587e2c657e4d938ce0a9f9b65501327",
"9838c215f3adff337d76ed2833d4422e",
"9ae47ca6036fb6f0ff1a5bba3983ffed",
"9b2b3b421cb78d8a79027ae89bb306f3",
"9bd7b7a63fa26d8402c5a203c70410a7",
"9c3715767177310f52ae08016b240530",
"9eb70ae4ce570f75a9ee59f39e0951e5",
"a1536081d9fd8608da3325ca4ee9905e",
"a3b04bf19e475f7125140e54c3ef8f74",
"ac10e381c17ac61b6ed822d9d216ae2c",
"cd72451891a553a4043ea8e5b657047e",
"cee374a04a97803f533c8c0f8d96e18e",
"d0ac4556f27fa33995578444a4a48c9a",
"d72f7805cd378983bc591a751f5a59ee",
"dd59ef42eb7f7ba455349e3d055f9009",
"e1e83ef0c191df696eeac035b85f01da",
"e441573f5cebdaffbfba4ca1d31dda51",
"e527e0b59433177b40e1a6d3f63dbd45",
"e7d16968a99c3bb7434868747fd82be8",
"ef7f645925ab8eb02075832ca3133033",
"f1389e44c90189e64c46e7835bc12021",
"f3aff216ae60e8872b045b061cd7ec91",
"f4a6ef879821e31b2e1f8ab30f1e0379",
"fa64d29a142b33ffec1e11d1a31aff99",
"fda1b25645be6f69bbed7aac3add2cb8",
"ff28f39727c9266eacd10e86e85170be",

]


In [3]:
images = []
for s in interesting:
    filename = '../jobs/results/{}/images.npz'.format(s)
    images.append(preprocess_gen_data(joblib.load(filename))[0:500])

In [5]:
X = np.concatenate(images, axis=0)

In [ ]:
S = X.copy()
S[:, 0, 0, 0:]=0.5
S[:, 0, -1, 0:]=0.5
S[:, 0, 0:, 0]=0.5
S[:, 0, 0:, -1]=0.5
R = X.copy()
R = encode(R)
R = R.reshape((X.shape[0], -1))

In [ ]:
model = load_model('../jobs/results/1b5f929796b52352a009ab37f602bfbf/model.pkl')

In [ ]:
model, data, layers = model

In [ ]:
import theano
import lasagne.layers as L
import theano.tensor as T
xt = T.tensor4()
encode = theano.function([xt], L.get_output(layers['hid5'], xt))

In [6]:
#R = encode(X)
R = X.reshape((X.shape[0], -1))

In [8]:
print(R.shape)

(43500, 50)


In [ ]:
clus = KMeans(n_clusters=1000)
clus.fit(R[0:10000])

In [ ]:
R_clus = (clus.predict(R)==240)
img = disp_grid(X[R_clus][0:16], border=1, bordercolor=(0.3, 1, 1))
img = 1 - img
imsave('out_scatter.png', img)
Image('out_scatter.png')

In [10]:
from tools.viz.image_scatter import image_scatter
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
R = PCA(n_components=50).fit_transform(R)
#R= TSNE(perplexity=15).fit_transform(R)
#img = image_scatter(R, 0, 40, res=4000)


imsave('out_scatter.png', img)
Image('out_scatter.png')

TypeError: 'int' object is not iterable